In [183]:
import pandas as pd
import numpy as np

from catboost import CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from datetime import datetime

pd.set_option('display.max_rows', 5000)

zhi_df2=pd.read_csv('../data/Zip_Zri_MultiFamilyResidenceRental.csv')


zri = pd.melt(zhi_df2, id_vars = zhi_df2.columns[0:6], value_vars = zhi_df2.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionName', 'year-month'])

#zri=zri[zri['year-month']>'2014-02']

zri=zri.fillna(0)
zri['gapsize']=zri['ZRI'].rolling(6).sum()
droplist=list(zri[(zri['gapsize']==0) & (zri['year-month']>'2014-06')]['RegionName'].drop_duplicates())
zri[(zri['gapsize']==0)].shape[0]/zri.shape[0]
len(droplist)/len(zri['RegionName'].drop_duplicates())


0.30037614185921546

In [225]:
zri2 = pd.melt(zhi_df2, id_vars = zhi_df2.columns[0:6], value_vars = zhi_df2.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionName', 'year-month'])

zri2[(zri2['year-month']=='2019-12') & (zri2['ZRI'].isna())]=zri2[(zri2['year-month']=='2019-12') & (zri2['ZRI'].isna())].fillna(method='ffill')

zri2=zri2.groupby('RegionName').apply(lambda group: group.interpolate())
#zri2[zri2['ZRI'].isna()]

#zri2['ZRI']=(zri2['ZRI']/zri2['ZRI'].shift(1))-1

zri2=zri2[(zri2['year-month']>'2014-01') & (zri2['year-month']<'2020-01')]

zri3=zri2[~(zri2['RegionName'].isin(droplist))]

zri3['ZRI']=zri3['ZRI'].fillna(method='bfill')


<ipython-input-225-ed144cebc724>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zri3['ZRI']=zri3['ZRI'].fillna(method='bfill')


In [227]:
#zip codes to drop
len(droplist)

559

In [228]:
#remaining filterd ZRI data:
len(zri3['RegionName'].drop_duplicates())

1302

In [226]:
zri3.isna().sum()

RegionID       0
RegionName     0
City           0
State          0
Metro         70
CountyName     0
year-month     0
ZRI            0
dtype: int64

## 